In [3]:
import gzip
import string
from collections import defaultdict
import matplotlib.pyplot as plt
from IPython.display import clear_output
import string
import random
import math

from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
import sklearn.preprocessing

In [25]:
def parse(file):
#     total_count = 0
    keep = 0
    withdraw = 0
    for l in gzip.open(file):
        l = l.replace(b'true', b'True') # all data read from the file is returned as bytes
        l = l.replace(b'false', b'False')
        item = eval(l)
        year = int(item['reviewTime'].split(',')[1])
        if year > 2010 and item['verified'] and 'reviewText' in item.keys():
            keep += 1
            
            yield item
        else:
            withdraw += 1
            
        clear_output(wait=True)
        print(keep+withdraw, keep, withdraw)
        if keep == 140000: break
        

In [26]:
dataset = list(parse("Electronics_5.json.gz"))

194963 140000 54963


In [27]:
print(dataset[100000])

{'overall': 5.0, 'verified': True, 'reviewTime': '10 31, 2016', 'reviewerID': 'A1714OIHN8BLOD', 'asin': 'B00007KDX6', 'style': {'Color:': ' Silver'}, 'reviewerName': 'Angelo S.', 'reviewText': 'Fast ship, No problems..', 'summary': 'Five Stars', 'unixReviewTime': 1477872000}


In [28]:
print(len(dataset))

140000


In [6]:
rating_count = defaultdict(int)
reviewer = defaultdict(int)
review_count_year = defaultdict(int) # number of reviews in each year
asin = set()
ave_review_length = 0
ave_rating = 0
ave_rating_year = defaultdict(int)
count_reviewText = 0

# rating count
for d in dataset:
    rating_count[d['overall']] += 1
    ave_rating += d['overall']
    if 'reviewText' in d.keys(): # not every item has key 'reviewText'
        count_reviewText += 1
        ave_review_length += len(d['reviewText']) 
    reviewer[d['reviewerID']] += 1
    asin.add(d['asin'])
    
    # number of reviews in each year
    year = d['reviewTime'].split(',')[1].strip()
    review_count_year[year] += 1
    
    ave_rating_year[year] += d['overall']
    
# count average year rating 
for k in ave_rating_year.keys():
    ave_rating_year[k] = ave_rating_year[k]/review_count_year[k]
    
# sort dictionay
sorted_reviewer = [(r, c) for r, c in sorted(reviewer.items(), key = lambda i:i[1], reverse = True)]
sorted_review_count_year = [(y,n) for y,n in sorted(review_count_year.items(), key = lambda i:int(i[0]))]
sorted_ave_rating_year = [(y,r) for y,r in sorted(ave_rating_year.items(), key = lambda i:int(i[0]))]

# average rating
ave_rating = ave_rating / len(dataset)
ave_review_length = ave_review_length / len(dataset)

print("Rating count:", rating_count.items())
print("Average rating:", ave_rating)
print("Number of reviewer:", len(reviewer))
print("Number of music:", len(asin))
print("Average review length:", ave_review_length)
print("reviewText:", count_reviewText)
print("Number of reviews in each year", sorted_review_count_year)
print("Average rating of each year:", sorted_ave_rating_year)

Rating count: dict_items([(3.0, 19767), (4.0, 48140), (5.0, 207739), (1.0, 14411), (2.0, 9943)])
Average rating: 4.416176666666667
Number of reviewer: 197759
Number of music: 5723
Average review length: 237.69216333333333
reviewText: 300000
Number of reviews in each year [('2011', 16757), ('2012', 22388), ('2013', 49832), ('2014', 60338), ('2015', 68950), ('2016', 48057), ('2017', 27601), ('2018', 6077)]
Average rating of each year: [('2011', 4.2492689622247415), ('2012', 4.320796855458282), ('2013', 4.390471985872532), ('2014', 4.427624382644436), ('2015', 4.468440899202321), ('2016', 4.460474020434068), ('2017', 4.410927140321003), ('2018', 4.405463221984532)]


In [46]:
#positive words and negative words

positive_words = defaultdict(int)
negative_words = defaultdict(int)

# review with 'will' and 'if'
target_list = []
# punctuation = string.punctuation
for d in dataset:
    in_target = False
    words = ''.join([c for c in d['reviewText'] if c not in string.punctuation])
    if d['overall'] >= 4: 
        for w in words.split():
            positive_words[w] += 1
    else:
        for w in words.split():
            negative_words[w] += 1
            if (w == 'if' or w == 'will' or w == 'when') and not in_target:
                target_list.append(d['reviewText'])
                in_target = True
                

positive_words_list = [(w, c/len(positive_words)) for w, c in sorted(positive_words.items(), key = lambda i:i[1], reverse = True)]
negative_words_list = [(w, c/len(negative_words)) for w, c in sorted(negative_words.items(), key = lambda i:i[1], reverse = True)]


### Data Preprocessing

In [29]:
random.shuffle(dataset)

In [68]:
f = open('assignment2_dataset.text', 'w')
c = 0
for d in dataset:
    f.write(str(d) + '\n')
    c+= 1
    if c%100 == 0:
        clear_output(wait=True)
        print(c)
print('finish store')

        
# train_set = dataset[:100000] 
# validation_set = dataset[100000:120000] 
# test_set = dataset[120000:] 

140000
finish store


In [4]:
dataset = []
c = 0
for l in open('assignment2_dataset.text'):
# for l in f.readline():
    c +=1
    dataset.append(eval(l))
    if c%100 == 0:
        clear_output(wait=True)
        print(c)
clear_output(wait=True)
print(c)

138330


In [5]:
train_set = dataset[:100000] 
validation_set = dataset[100000:120000] 
test_set = dataset[120000:] 

### Features extraction

In [6]:
# unigram and bigram 
unigram = defaultdict(int)
bigram = defaultdict(int)
for d in train_set:
    words = d['reviewText'].replace('\n', '').strip()
    words = ''.join([c for c in words if c not in string.punctuation])
    words = words.split()
    for i in range(len(words)):
        unigram[words[i]] += 1
        
    for i in range(len(words)-1):
        bigram[words[i] + ' ' + words[i+1]] += 1

unigram_list = [w for w in sorted(unigram.items(), key = lambda i:i[1], reverse = True)]
bigram_list = [w for w in sorted(bigram.items(), key = lambda i:i[1], reverse = True)]

unigram_top_words = [w[0] for w in unigram_list[:1000]]
bigram_top_words = [w[0] for w in bigram_list[:1000]]

unigram_top_wordsId = dict(zip(unigram_top_words, range(len(unigram_top_words))))
bigram_top_wordsId = dict(zip(bigram_top_words, range(len(bigram_top_words))))

In [38]:
print(unigram_top_words[:10])
print(bigram_top_words[:10])

['the', 'I', 'and', 'to', 'a', 'it', 'for', 'is', 'of', 'my']
['of the', 'I have', 'in the', 'on the', 'for the', 'is a', 'and the', 'to the', 'with the', 'for a']


In [7]:
# idf
unigram_idf = [math.log(100000/x[1], 10) for x in unigram_list[:1000]]
bigram_idf = [math.log(100000/x[1], 10) for x in bigram_list[:1000]]

In [8]:
# data preprocessing #unigram #bigram

X_train_uni, y_train = [], []
X_train_bi = []

c = 0
for d in train_set:
    c += 1
    #unigram
    feat = []
    for w in unigram_top_words:
        feat.append(d['reviewText'].count(w))
    feat.append(1)
    X_train_uni.append(feat)
    y_train.append(d['overall'])
    
    #bigram
    feat = []
    for w in bigram_top_words:
        line = d['reviewText'].replace('\n', '')
        for p in string.punctuation:
            line = line.replace(p, '')
        feat.append(line.count(w))
    feat.append(1)
    X_train_bi.append(feat)
    
    if c % 10 == 0:
        clear_output(wait=True)
        print("Processing progress:", c ,"/100000")


Processing progress: 1820 /100000


KeyboardInterrupt: 

In [10]:
# data preprocessing
X_train_uni_tfidf, X_train_bi_tfidf = [], []
c = 0
for d in train_set:
    c += 1
    feat = []
    for i in range(len(unigram_top_words)):
        tfidf = d['reviewText'].count(unigram_top_words[i]) * unigram_idf[i]
        feat.append(tfidf)
    feat.append(1)
    X_train_uni_tfidf.append(feat)
    
    #bigram
    feat = []
    for i in range(len(bigram_top_words)):
        line = d['reviewText'].replace('\n', '')
        for p in string.punctuation:
            line = line.replace(p, '')
        tfidf = line.count(bigram_top_words[i]) * bigram_idf[i]
        feat.append(tfidf)
    feat.append(1)
    X_train_bi_tfidf.append(feat)
    
    if c % 10 == 0:
        clear_output(wait=True)
        print("Processing progress:", c ,"/100000")



Processing progress: 100000 /100000


In [12]:
y_train = []
for d in train_set:
    y_train.append(d['overall'])

In [92]:
#normalization
X_train_uni = sklearn.preprocessing.normalize(X_train_uni)
X_train_bi = sklearn.preprocessing.normalize(X_train_bi)

### Evaluation Method

In [17]:
# accuracy
def accuracy(y_pred, y):
    return sum([True for i in range(len(y)) if y_pred[i] == y[i]])/len(y)

In [ ]:
#confusion matrix

### MODEL

#### 1. KNN

In [98]:
#KNN #unigram
knn_uni = KNeighborsClassifier(n_neighbors=5)
knn_uni.fit(X_train_uni, y_train)

y_pred = knn_uni.predict(X_train_uni[14000:15000])
print(accuracy(y_pred,y_train[14000:15000]))

0.754


In [41]:
#KNN #bigram
knn_bi = KNeighborsClassifier(n_neighbors=5)
knn_bi.fit(X_train_bi, y_train)

y_pred = knn_bi.predict(X_train_bi[14000:15000])
print(accuracy(y_pred,y_train[14000:15000]))

0.754
0.74


In [15]:
#KNN #unigram #tfidf
knn_uni_tfidf = KNeighborsClassifier(n_neighbors=5)
knn_uni_tfidf.fit(X_train_uni_tfidf, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [18]:
y_pred = knn_uni_tfidf.predict(X_train_uni_tfidf[14000:15000])
print(accuracy(y_pred,y_train[14000:15000]))

0.757


In [19]:
#KNN #bigram #tfidf
knn_bi_tfidf = KNeighborsClassifier(n_neighbors=5)
knn_bi_tfidf.fit(X_train_bi_tfidf, y_train)


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [21]:
y_pred = knn_bi_tfidf.predict(X_train_bi_tfidf[14000:15000])
print(accuracy(y_pred,y_train[14000:15000]))

0.748


### 2. Logistic Regress

In [110]:
# Logistic Regress #unigram
clf_uni = LogisticRegression(C=100, solver = 'saga')
clf_uni.fit(X_train_uni, y_train)
y_pred = clf_uni.predict(X_train_uni[14000:15000])
print(accuracy(y_pred,y_train[14000:15000]))

0.733


/home/l1lu/.local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [94]:
# Logistic Regress #bigram
clf_bi = LogisticRegression(C=10, class_weight='balanced')
clf_bi.fit(X_train_bi, y_train)
y_pred = clf_bi.predict(X_train_bi[14000:15000])
print(accuracy(y_pred,y_train[14000:15000]))

0.725


/home/l1lu/.local/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [99]:
# Logistic Regress #tfidf


### 3. SVM

In [ ]:
# SVM #unigram
lin_clf_uni = svm.LinearSVC(C=10)
lin_clf_uni.fit(X_train_uni, y_train)

In [ ]:
y_pred = lin_clf_uni.predict(X_train_uni[14000:15000])
print(accuracy(y_pred, y_train[14000:15000]))

In [104]:
# SVM #bigram
lin_clf_bi = svm.LinearSVC()
lin_clf_bi.fit(X_train_bi, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [105]:
y_pred = lin_clf_bi.predict(X_train_bi[14000:15000])
print(accuracy(y_pred, y_train[14000:15000]))

0.73
